# Check Census data and compile, clean, convert to the right units
* Census API doesn't seem to be working properly...check that all years have the same number of obs

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
for name in ['commute', 'income', 'income_range', 'vehicles', 'tenure', 'race']:
    file_name = f'{name}_tract'
    df = pd.read_csv(f'../data/{file_name}.csv')
    display(name)
    display(df.year.value_counts().sort_index())

'commute'

2010    70380
2011    70380
2012    70380
2013    70380
2014    70380
2015    70380
2016    70380
2017    70380
2018    70380
Name: year, dtype: int64

'income'

2010    46920
2011    46920
2012    46920
2013    46920
2014    46920
2015    46920
2016    46920
2017    46920
2018    46920
Name: year, dtype: int64

'income_range'

...

2010    46920
2011    46920
2012    46920
2013    46920
2014    46920
2015    46920
2016    46920
2017    46920
2018    46920
Name: year, dtype: int64

'tenure'

2010    7038
2011    7038
2012    7038
2013    7038
2014    7038
2015    7038
2016    7038
2017    7038
2018    7038
Name: year, dtype: int64

'race'

2010    18768
2011    18768
2012    18768
2013    18768
2014    18768
2015    18768
2016    18768
2017    18768
2018    18768
Name: year, dtype: int64

## Income

In [3]:
full_df = pd.DataFrame()

for name in ['commute', 'income', 'income_range', 'vehicles', 'tenure', 'race']:
    file_name = f'{name}_tract'
    df = pd.read_csv(f'../data/{file_name}.csv', dtype={"GEOID": "str"})
    df = df[['GEOID', 'variable', 'estimate', 'year']]
    df['GEOID'] = df.GEOID.str.pad(width = 11, side = 'left', fillchar = '0')
    full_df = full_df.append(df, sort = False)
    

full_df.to_parquet('../data/raw_census.parquet')

In [4]:
df = pd.read_parquet('../data/raw_census.parquet')

In [5]:
geoid = ['06037101110']
y = 2010
df = df[df.GEOID.isin(geoid) & (df.year==y)]

In [6]:
acs_tables = {
    'S1903': 'income', 
    'B19001': 'incomerange',
    'S0801': 'commute',
    'S0802': 'vehicles',
    'B25008': 'tenure',
    'B02001': 'race',
}

In [7]:
def tag_acs_table(df):
    pattern = re.compile('([A-Za-z0-9]+)_')
    
    df['table'] = df.apply(
        lambda row: acs_tables.get(pattern.match(row.variable).group(1)),
        axis = 1
    )
    
    # Find the other B19001A, B19001B, etc tables and tag them
    df['table'] = df.apply(
        lambda row: 'incomerange' if 'B19001' in row.variable else row.table,
        axis = 1
    )
    
    return df

In [8]:
df = tag_acs_table(df)

In [9]:
df.table.value_counts()

incomerange    170
commute         30
vehicles        20
income          20
race             8
tenure           3
Name: table, dtype: int64

In [10]:
# Tag main variable
def income_vars(row):
    if '_C01' in row.variable:
        return 'hh'
    elif '_C02' in row.variable:
        return 'medincome'
    elif '_C03' in row.variable:
        return 'medincome'
    
def incomerange_vars(row):
    if 'B19001_' in row.variable:
        return 'total'
    elif 'B19001A' in row.variable:
        return 'white'
    elif 'B19001B' in row.variable:
        return 'black'
    elif 'B19001C' in row.variable:
        return 'amerind'
    elif 'B19001D' in row.variable:
        return 'asian'
    elif 'B19001E' in row.variable:
        return 'pacis'
    elif 'B19001F' in row.variable:
        return 'other'
    elif 'B19001G' in row.variable:
        return 'race2'
    elif 'B19001H' in row.variable:
        return 'nonhisp'
    elif 'B19001I' in row.variable:
        return 'hisp'
    
main_vars_dict = {
    'income': income_vars,
    'incomerange': incomerange_vars,

}

In [12]:
# Secondary variable - use last 2 characters
income = {'01': 'total', '02': 'white', '03': 'black', '04': 'amerind', '05': 'asian',
       '06': 'pacis', '07': 'other', '08': 'race2', '09': 'hisp', '10': 'nonhisp'}

incomerange = {'01': 'total', '02': 'lt10', '03': 'r10to14', '04': 'r15to19', '05': 'r20to24',
           '06': 'r25to29', '07': 'r30to34', '08': 'r35to39', '09': 'r40to44', '10': 'r45to49',
           '11': 'r50to59', '12': 'r60to74', '13': 'r75to99', '14': 'r100to124', '15': 'r125to149',
           '16': 'r150to199', '17': 'gt200'}

In [14]:
def tag_main_secondary_variable(df):
    df['main_var'] = df.apply(lambda row: main_vars_dict[row['table']](row), axis = 1)
    
    df['last2'] = df.variable.str[-3:-1]

    def pick_secondary_var(row):
        if row.table=='income':
            return income[row.last2]
        elif row.table=='incomerange':
            return incomerange[row.last2]
    
    df['second_var'] = df.apply(pick_secondary_var, axis = 1)
    
    return df

In [15]:
df = tag_main_secondary_variable(df)

KeyError: ('commute', 'occurred at index 0')